In [115]:
import pandas as pd
import re

dados_portugues = pd.read_csv("data/stackoverflow_portugues.csv")
dados_portugues.head()

questao_portugues = dados_portugues["Questão"][5]
print(questao_portugues)

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre><code>//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
</code></pre>

<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>

<pre><code>insert into Produtos (coluna) values(1); DROP TABLE Produtos;
</code></pre>

<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



In [116]:
dados_ingles = pd.read_csv("data/stackoverflow_ingles.csv")
dados_espanhol = pd.read_csv("data/stackoverflow_espanhol.csv", 
                             sep=";", encoding="cp1252")


# Testing Regex

In [117]:
re.findall(r"<.*?>", questao_portugues)

['<p>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<p>',
 '</p>']

In [118]:
re.sub(r"<.*?>", "", questao_portugues)

"Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:\n\n//----CONSULTA SQL----//\n$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');\n\n\nLogo, digamos que o usuário usar a sentença: 1); DROP TABLE Produtos; para ao campo valor o comando ficaria: \n\ninsert into Produtos (coluna) values(1); DROP TABLE Produtos;\n\n\nEle vai inserir um novo registro cujo o campo coluna será 1 e logo em seguida ele vai deletar a tabela Produtos.\n\nComo posso melhorar meu código para prevenir essa situação?\n"

In [119]:
regex = re.compile(r"70")
regex.search("684746216489798770")

<re.Match object; span=(16, 18), match='70'>

In [120]:
def remover(textos, regex):
    if type(textos) == str:
        return regex.sub("", textos)
    else:
        return [regex.sub("", texto) for texto in textos]
    
def substituir_codigo(textos, regex):
    if type(textos) == str:
        return regex.sub("CODE", textos)
    else:
        return [regex.sub("CODE", texto) for texto in textos]

In [121]:
regex_HTML = re.compile(r"<.*?>")
regex_code = re.compile(r"<code>(.|\n)*?</code>")

In [122]:
questao_portugues_sem_tag = remover(questao_portugues, regex_HTML)

In [123]:
print(substituir_codigo(questao_portugues, regex_code))

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre>CODE</pre>

<p>Logo, digamos que o usuário usar a sentença: CODE para ao campo CODE o comando ficaria: </p>

<pre>CODE</pre>

<p>Ele vai inserir um novo registro cujo o campo CODE será CODE e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



# Data Manip With Regex

In [124]:
regex_HTML = re.compile(r"<.*?>")
regex_code = re.compile(r"<code>(.|\n)*?</code>")
regex_pontuacao = re.compile(r"[^[\w\s]]")
regex_digitos = re.compile(r"\d+")
regex_espaco = re.compile(r" +")
regex_quebra_linha = re.compile(r"(\n)")

def substituir(textos, regex, subs=""):
    if type(textos) == str:
        return regex.sub(subs, textos)
    else:
        return [regex.sub(subs, texto) for texto in textos]
    
def minusculo(textos):
    if type(textos) == str:
        return textos.lower()
    else:
        return [texto.lower() for texto in textos]
    
def add_preprocessed_column(data):
    questoes_sem_code = substituir(data["Questão"], regex_code, "CODE")
    questoes_sem_code_tag = substituir(questoes_sem_code, regex_HTML)
    data["sem_code_tag"] = questoes_sem_code_tag

    questoes_sem_pont = substituir(questoes_sem_code_tag, regex_pontuacao)
    questoes_sem_pont_min = minusculo(questoes_sem_pont)
    questoes_sem_pont_min_dig = substituir(questoes_sem_pont_min, regex_digitos)
    questoes_sem_quebra_linha = substituir(questoes_sem_pont_min_dig, 
                                           regex_quebra_linha, " ")
    questoes_sem_espaco_duplicado = substituir(questoes_sem_quebra_linha, 
                                               regex_espaco, " ")
    
    
    data["dados_tratados"] = questoes_sem_espaco_duplicado
    return data


In [125]:
dados_portugues = add_preprocessed_column(dados_portugues)
dados_ingles = add_preprocessed_column(dados_ingles)
dados_espanhol = add_preprocessed_column(dados_espanhol)

In [126]:
dados_portugues

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,dados_tratados
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...,qual é a diferença entre code e code? podem me...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...,uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...,é comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...,me parecem termos muito próximos e eventualmen...
...,...,...,...,...,...,...,...,...
495,194857,O que é Polyfill?,<p>Já vi esse termo <em>Polyfill</em> sendo ut...,<javascript><terminologia><polyfill>,26,6860,Já vi esse termo Polyfill sendo utilizado vári...,já vi esse termo polyfill sendo utilizado vári...
496,323137,Pra que serve o comando LOCK TABLES?,<p>Esses dias me deparei com um trecho de um S...,<mysql>,26,657,Esses dias me deparei com um trecho de um SQL ...,esses dias me deparei com um trecho de um sql ...
497,232958,O que é um valor opaco?,<p>Por vezes vejo em documentações ou especifi...,<nomenclatura>,26,587,Por vezes vejo em documentações ou especificaç...,por vezes vejo em documentações ou especificaç...
498,227907,"O que são Proxy, Gateway e Tunnel no protocolo...","<p>Na especificação do protocolo HTTP, mais pr...",<http>,26,625,"Na especificação do protocolo HTTP, mais preci...","na especificação do protocolo http, mais preci..."


# Natural Language Model

In [127]:
from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends

texto_teste = "alura"
list(bigrams(pad_both_ends(texto_teste, n=2)))

[('<s>', 'a'), ('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a'), ('a', '</s>')]

### Adding Labels

In [128]:
dados_portugues["idioma"] = "port"
dados_ingles["idioma"] = "ing"
dados_espanhol["idioma"] = "esp"

# Creating Model - MLE

In [129]:
from sklearn.model_selection import train_test_split

port_treino, port_teste = train_test_split(dados_portugues["dados_tratados"],
                                           test_size=0.2, random_state=42)
esp_treino, esp_teste = train_test_split(dados_espanhol["dados_tratados"],
                                           test_size=0.2, random_state=42)
ing_treino, ing_teste = train_test_split(dados_ingles["dados_tratados"],
                                           test_size=0.2, random_state=42)                                           

In [130]:
from nltk.tokenize import WhitespaceTokenizer

todas_questoes_port = ' '.join(port_treino)

todas_palavras_port = WhitespaceTokenizer().tokenize(todas_questoes_port)

The method padded_everygram_pipeline already separates the data into the informed number of ngrams and adds the fake chars in every setence. It also returns two values that can be directly used in nltk models. 

In [131]:
from nltk.lm.preprocessing import padded_everygram_pipeline

port_treino_bigram, vocab_port = padded_everygram_pipeline(2, todas_palavras_port)

In [132]:
from nltk.lm import MLE

modelo_port = MLE(2)

modelo_port.fit(port_treino_bigram, vocab_port)

In [133]:
def treinar_modelo_mle(lista_textos):
    todas_questoes = ' '.join(lista_textos)

    todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
    bigrams, vocab = padded_everygram_pipeline(2, todas_palavras)
    
    modelo = MLE(2)
    modelo.fit(bigrams, vocab)

    return modelo

In [134]:
modelo_port = treinar_modelo_mle(port_treino)
modelo_ing = treinar_modelo_mle(ing_treino)
modelo_esp = treinar_modelo_mle(esp_treino)

In [135]:
def calcular_perplexidade(modelo, texto):
    perplexidade = 0
    
    palavras = WhitespaceTokenizer().tokenize(texto)
    palavras_fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in palavras]
    palavras_bigrams = [list(bigrams(palavra)) for palavra in palavras_fakechar]

    for palavra in palavras_bigrams:
        perplexidade += modelo.perplexity(palavra)

    return perplexidade

In [136]:
calcular_perplexidade(modelo_ing, "good morning")

30.867120442692816

In [137]:
calcular_perplexidade(modelo_port, port_teste.iloc[0])

inf

### Laplace's Model

In [138]:
from nltk.lm import Laplace

def treinar_modelo_laplace(lista_textos):
    todas_questoes = ' '.join(lista_textos)

    todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
    bigrams, vocab = padded_everygram_pipeline(2, todas_palavras)
    
    modelo = Laplace(2)
    modelo.fit(bigrams, vocab)

    return modelo

In [139]:
modelo_port_laplace = treinar_modelo_laplace(port_treino)
modelo_ing_laplace = treinar_modelo_laplace(ing_treino)
modelo_esp_lapalce = treinar_modelo_laplace(esp_treino)

In [142]:
calcular_perplexidade(modelo_port_laplace, port_teste.iloc[0])

852.1388896944829

# Model To That Predicts Language from Text

In [147]:
def atribui_idioma(lista_textos):
    
    idioma = []
    for texto in lista_textos:
        portugues = calcular_perplexidade(modelo_port, texto)
        ingles = calcular_perplexidade(modelo_ing, texto)
        espanhol = calcular_perplexidade(modelo_esp, texto)

        if ingles >= portugues <= espanhol:
            idioma.append("portugues")
        elif portugues > ingles < espanhol:
            idioma.append("ingles")
        else:
            idioma.append("espanhol")

    return idioma

In [153]:
resultados_portugues = atribui_idioma(port_teste)
resultados_portugues.count("portugues")

100

In [154]:
resultados_ingles = atribui_idioma(ing_teste)
resultados_ingles.count("ingles")

81

In [155]:
resultados_espanhol = atribui_idioma(esp_teste)
resultados_espanhol.count("espanhol")

71